In [1]:
#Import relevant libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Loading the dreams dataset
df = pd.read_csv('./data/dreams_raw_dataset.csv')

In [3]:
#Dataset preview
df.head()

,date_of_birth,date_of_enrollment,Agency,implementing_partner_name,county,head_of_household,head_of_household_other,age_of_household_head,father_alive,mother_alive,...,intervention_name,intervention_date,result,bio_medical,social_protection,behavioral,post_gbv_care,other_interventions,exit_age,exit_reason_other
0,9/16/2004,2/22/2020,USAID,USAID Tumikia Mtoto,Nairobi,Mother,NaN,58.0,Yes,Yes,...,HTS - HTS (Client),8/8/2020,Negative,1,1,1,0,0,NaN,NaN
1,8/2/2004,12/18/2019,USAID,USAID Tumikia Mtoto,Nairobi,Father,NaN,43.0,Yes,Yes,...,HTS - HTS (Client),4/27/2020,Negative,1,1,1,0,0,NaN,NaN
2,10/20/2005,3/7/2020,USAID,USAID Tumikia Mtoto,Nairobi,Mother,NaN,41.0,No,Yes,...,HTS - HTS (Client),8/12/2020,Negative,1,1,0,0,0,NaN,NaN
3,1/18/2006,3/3/2020,USAID,USAID Tumikia Mtoto,Nairobi,Mother,NaN,45.0,No,Yes,...,HTS - HTS (Client),8/12/2020,Negative,1,1,1,0,0,NaN,NaN
4,3/22/2004,12/18/2019,USAID,USAID Tumikia Mtoto,Nairobi,Father,NaN,42.0,Yes,Yes,...,HTS - HTS (Client),4/27/2020,Negative,1,1,1,0,0,NaN,NaN


In [4]:
df.columns

Index(['date_of_birth', 'date_of_enrollment', 'Agency',
       'implementing_partner_name', 'county', 'head_of_household',
       'head_of_household_other', 'age_of_household_head', 'father_alive',
       'mother_alive', 'are_both_parents_alive', 'is_one_parent_alive',
       'parents_alive_status', 'no_of_people_in_household', 'no_of_females',
       'no_of_males', 'no_of_adults', 'no_of_children',
       'ever_missed_full_day_food_in_4wks', 'no_of_days_missed_food_in_4wks',
       'currently_in_school', 'current_school_level', 'current_income_source',
       'ever_tested_for_hiv', 'last_test_result', 'ever_had_sex',
       'age_at_first_sexual_encounter', 'has_sexual_partner',
       'age_of_last_partner', 'used_condom_with_last_partner',
       'dreams_program', 'dreams_program_other', 'exit_status', 'exit_reason',
       'intervention_name', 'intervention_date', 'result', 'bio_medical',
       'social_protection', 'behavioral', 'post_gbv_care',
       'other_interventions', 'exit_a

In [5]:
cols= ['head_of_household_other','is_one_parent_alive','parents_alive_status','dreams_program_other','exit_status','exit_reason',
'intervention_name','intervention_date','other_interventions','exit_reason_other','no_of_days_missed_food_in_4wks','Agency','date_of_enrollment']

In [6]:
df1 = df.drop(columns=cols)

In [7]:
df1.shape

(455807, 31)

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age_of_household_head,434789.0,2074.593380,1.106761e+06,-48.0,33.0,42.0,50.0,727284575.0
no_of_people_in_household,448607.0,5.774555,3.837220e+00,0.0,4.0,6.0,7.0,356.0
no_of_females,448475.0,3.330979,2.412982e+00,0.0,2.0,3.0,4.0,216.0
no_of_males,448125.0,2.443664,1.911644e+00,0.0,1.0,2.0,3.0,140.0
no_of_adults,448328.0,2.492278,1.295561e+00,0.0,2.0,2.0,3.0,77.0
no_of_children,448034.0,3.283115,3.433194e+00,0.0,2.0,3.0,4.0,328.0
age_at_first_sexual_encounter,226256.0,16.781363,2.099138e+01,0.0,15.0,17.0,18.0,4118.0
bio_medical,455807.0,0.990619,9.640113e-02,0.0,1.0,1.0,1.0,1.0
social_protection,455807.0,0.988105,1.084155e-01,0.0,1.0,1.0,1.0,1.0
behavioral,455807.0,0.882301,3.222517e-01,0.0,1.0,1.0,1.0,1.0


In [9]:
df1.describe(include='object').T

,count,unique,top,freq
date_of_birth,455807,7839,1/1/2000,1011
implementing_partner_name,455807,20,Afya Ziwani,101893
county,446551,7,Nairobi,100411
head_of_household,446192,8,Father,173119
father_alive,442399,3,Yes,295841
mother_alive,442820,3,Yes,378897
are_both_parents_alive,455807,2,Yes,273198
ever_missed_full_day_food_in_4wks,448640,2,Yes,257179
currently_in_school,451634,2,Yes,263268
current_school_level,254941,6,Secondary Level,132952


In [10]:
df1.isna().sum()

date_of_birth                             0
implementing_partner_name                 0
county                                 9256
head_of_household                      9615
age_of_household_head                 21018
father_alive                          13408
mother_alive                          12987
are_both_parents_alive                    0
no_of_people_in_household              7200
no_of_females                          7332
no_of_males                            7682
no_of_adults                           7479
no_of_children                         7773
ever_missed_full_day_food_in_4wks      7167
currently_in_school                    4173
current_school_level                 200866
current_income_source                129564
ever_tested_for_hiv                       0
last_test_result                          0
ever_had_sex                           7727
age_at_first_sexual_encounter        229551
has_sexual_partner                   229506
age_of_last_partner             

In [11]:
df1['result'].isna().sum()

0

In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455807 entries, 0 to 455806
Data columns (total 31 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   date_of_birth                      455807 non-null  object 
 1   implementing_partner_name          455807 non-null  object 
 2   county                             446551 non-null  object 
 3   head_of_household                  446192 non-null  object 
 4   age_of_household_head              434789 non-null  float64
 5   father_alive                       442399 non-null  object 
 6   mother_alive                       442820 non-null  object 
 7   are_both_parents_alive             455807 non-null  object 
 8   no_of_people_in_household          448607 non-null  float64
 9   no_of_females                      448475 non-null  float64
 10  no_of_males                        448125 non-null  float64
 11  no_of_adults                       4483

In [13]:
df1.duplicated().sum()

451

In [14]:
#check for duplicates and remove if any
df1 = df1.drop_duplicates()

In [15]:
df1.duplicated().sum()

0

## Modeling

In [16]:
#target and predictors
X = df1.drop('result', axis=1)
y = df1['result'].map({'Negative':0, 'Positive':1})

In [17]:
#train and test splits
X_train, X_test, y_train,y_test= train_test_split(X,y, test_size=0.2, random_state=42)

In [18]:
num_cols = ['age_of_household_head', 'no_of_people_in_household', 'no_of_females', 'no_of_males', 'no_of_adults', 'no_of_children',
 'age_at_first_sexual_encounter', 'bio_medical', 'social_protection', 'behavioral', 'post_gbv_care', 'exit_age']

cat_cols = ['date_of_birth', 'implementing_partner_name', 'county', 'head_of_household', 'father_alive', 'mother_alive',
 'are_both_parents_alive', 'ever_missed_full_day_food_in_4wks', 'currently_in_school', 'current_school_level', 'current_income_source',
 'ever_tested_for_hiv', 'last_test_result', 'ever_had_sex', 'has_sexual_partner', 'age_of_last_partner', 'used_condom_with_last_partner',
 'dreams_program']

In [19]:
# #separate categorical and numerical features
# num_cols = df1.select_dtypes(include=('int64', 'float64'))
# cat_cols = df1.select_dtypes(include='object')
# num_cols =num_cols.columns.tolist()
# cat_cols = cat_cols.columns.tolist()

In [20]:
print(type(cat_cols))

<class 'list'>


In [21]:
#Pipeline to clean, impute missing values
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with median
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with the most frequent category
    ('ohe', OneHotEncoder(handle_unknown='ignore')) 
])

#preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num',numerical_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])

In [22]:
#Baseline model as logistic regression
pipe_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf', LogisticRegression(class_weight='balanced'))
])

In [23]:
#Fit baseline model
pipe_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age_of_household_head',
                                                   'no_of_people_in_household',
                                                   'no_of_females',
                                                   'no_of_males',
                                                   'no_of_adults',
                                                   'no_of_children',
                                                   'age_at_first_sexual_encounter',
                                                   'bio_medical',
                                                   'social_protection',
                                                   'behavi...
                                                   'mother_alive',
                                                   'are_both_parents_alive',
                                                   'ever_missed_full_day_food_in_4wks',
                                                   'currently_in_school',
                                                   'current_school_level',
                                                   'current_income_source',
                                                   'ever_tested_for_hiv',
                                                   'last_test_result',
                                                   'ever_had_sex',
                                                   'has_sexual_partner',
                                                   'age_of_last_partner',
                                                   'used_condom_with_last_partner',
                                                   'dreams_program'])])),
                ('clf', LogisticRegression(class_weight='balanced'))])

In [24]:
#Make predictions
y_pred = pipe_model.predict(X_test)

In [25]:
# y_pred_binary = (y_pred > 0.5).astype(int)

In [26]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 90.95%


In [27]:
# confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[82780  8078]
 [  163    51]]


In [28]:
#Evaluate model performance

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")
print(f"F1 Score: {f1*100:.2f}%")

Precision: 0.63%
Recall: 23.83%
F1 Score: 1.22%


In [29]:
#Observation
#Classifier is heavily biased towards dominant class(negative class)
# Model performance is very low despite setting class_weight to 'balanced'

## Undersampling dominant class to match minority class

In [30]:
#balancing classes 
negative_class = df1[df1['result']=='Negative']
positive_class = df1[df1['result']=='Positive']

In [31]:
# Undersampling the majority negative class to match the size of the minority positive class
negative_class_undersampled = negative_class.sample(n=len(positive_class), random_state=42)

In [32]:
# Concatinate the undersampled negative class with the positive class
balanced_df = pd.concat([negative_class_undersampled, positive_class])

In [33]:
# Randomly shuffle the balanced dataset and reset index
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [34]:
# Separate predictor features and target
X_balanced = balanced_df.drop(columns=['result'])
y_balanced = balanced_df['result'].map({'Negative':0, 'Positive':1})

In [35]:
#train and test splits
X_train_balanced, X_test_balanced, y_train_balanced,y_test_balanced= train_test_split(X_balanced,y_balanced, test_size=0.2, random_state=42)

In [36]:
# Fit model 
pipe_model.fit(X_train_balanced, y_train_balanced)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age_of_household_head',
                                                   'no_of_people_in_household',
                                                   'no_of_females',
                                                   'no_of_males',
                                                   'no_of_adults',
                                                   'no_of_children',
                                                   'age_at_first_sexual_encounter',
                                                   'bio_medical',
                                                   'social_protection',
                                                   'behavi...
                                                   'mother_alive',
                                                   'are_both_parents_alive',
                                                   'ever_missed_full_day_food_in_4wks',
                                                   'currently_in_school',
                                                   'current_school_level',
                                                   'current_income_source',
                                                   'ever_tested_for_hiv',
                                                   'last_test_result',
                                                   'ever_had_sex',
                                                   'has_sexual_partner',
                                                   'age_of_last_partner',
                                                   'used_condom_with_last_partner',
                                                   'dreams_program'])])),
                ('clf', LogisticRegression(class_weight='balanced'))])

In [37]:
#Make predictions
y_pred_balanced = pipe_model.predict(X_test_balanced)

In [38]:
# confusion matrix for balanced dataset
cm_bal = confusion_matrix(y_test_balanced, y_pred_balanced)
print("Confusion Matrix:\n", cm_bal)

Confusion Matrix:
 [[150  60]
 [ 55 143]]


In [39]:
accuracy = accuracy_score(y_test_balanced, y_pred_balanced)
precision = precision_score(y_test_balanced, y_pred_balanced)
recall = recall_score(y_test_balanced, y_pred_balanced)
f1 = f1_score(y_test_balanced, y_pred_balanced)

print(f'Accuracy: {accuracy*100:.2f}%')
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")
print(f"F1 Score: {f1*100:.2f}%")

Accuracy: 71.81%
Precision: 70.44%
Recall: 72.22%
F1 Score: 71.32%
